In [8]:
root="/projects/ohlab/ruoyun/MECFS/train_model/MECFS"

kegg_table_patient="kegg_gene_patient_control_table_for_classification.txt"
metabolomics_table_patient="metabolomics_patient_control_table_for_classification.csv"
kegg_table_onset="kegg_gene_onset_table_for_classification.txt"
metabolomics_table_onset="metabolomics_onset_table_for_classification.csv"

abundance_table="metaphlan3_specie_filtered_zscored_071020.csv"
meta_table="MECFS_metadata_labels_062320.csv"

In [9]:
import pandas as pd
import numpy as np
import glob
import random

import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

In [11]:
#grid search
from sklearn.model_selection import RandomizedSearchCV

#save model
from joblib import dump, load

#validation
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import label_binarize

In [12]:
random_num = 1015

---

# import dataset here

In [13]:
rela_abun = pd.read_csv("%s/data/metagenomics/%s" %(root, abundance_table), 
                   index_col=0)
rela_abun_decode = rela_abun.iloc[:,:14]
rela_abun_data = rela_abun.iloc[:,14:]
print(rela_abun_decode.shape)
print(rela_abun_data.shape)

kegg_patient = pd.read_table("%s/result/feature_selection/%s" %(root, kegg_table_patient), 
                     sep='\t', index_col=0)
kegg_patient_decode = kegg_patient.iloc[:,:1]
kegg_patient_data = kegg_patient.iloc[:,1:]
print(kegg_patient_decode.shape)
print(kegg_patient_data.shape)

kegg_onset = pd.read_table("%s/result/feature_selection/%s" %(root, kegg_table_onset), 
                     sep='\t', index_col=0)
kegg_onset_decode = kegg_onset.iloc[:,:1]
kegg_onset_data = kegg_onset.iloc[:,1:]
print(kegg_onset_decode.shape)
print(kegg_onset_data.shape)

bioc_patient = pd.read_csv("%s/result/feature_selection/%s" %(root, metabolomics_table_onset), 
                   index_col=0)
bioc_patient_decode = bioc_patient.iloc[:,:12]
bioc_patient_data = bioc_patient.iloc[:,12:]
print(bioc_patient_decode.shape)
print(bioc_patient_data.shape)

bioc_onset = pd.read_csv("%s/result/feature_selection/%s" %(root, metabolomics_table_onset), 
                   index_col=0)
bioc_onset_decode = bioc_onset.iloc[:,:12]
bioc_onset_data = bioc_onset.iloc[:,12:]
print(bioc_onset_decode.shape)
print(bioc_onset_data.shape)

meta = pd.read_csv("%s/data/metadata/%s" %(root, meta_table), 
                   index_col=0)
meta_decode = meta.iloc[:, :4]
meta_data = meta.iloc[:, 4:]

print(meta_data.shape)

(169, 14)
(169, 224)
(1000, 1)
(1000, 224)
(1000, 1)
(1000, 224)
(400, 12)
(400, 184)
(400, 12)
(400, 184)
(6, 228)


# Patient vs Control

In [14]:
y_full = meta_data.loc['study_ptorhc', :]
model_list = load("%s/result/classification/patient_control_models_dictionary.joblib" %(root))

# abundance

In [15]:
X = rela_abun_data[:].transpose()
y = y_full[X.index].to_numpy()

In [16]:
ramdon_num_list = random.sample(range(1, 2000), 200)
imp_f_list = []
modle_list = []
    
for i in ramdon_num_list:
    ens = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                             learning_rate=0.05, loss='deviance', max_depth=5,
                             max_features=5, max_leaf_nodes=None,
                             min_impurity_decrease=0.0, min_impurity_split=None,
                             min_samples_leaf=3, min_samples_split=8,
                             min_weight_fraction_leaf=0.0, n_estimators=100,
                             n_iter_no_change=None, presort='deprecated',
                             random_state=i, subsample=0.8, tol=0.0001,
                             validation_fraction=0.1, verbose=0,
                             warm_start=False)
    modle_list.append(("%s" %(i), ens))
    ens.fit(X, y)
    imp = ens.feature_importances_
    imp_f_list.append(imp)
        
imp = pd.DataFrame(imp_f_list, columns=X.columns).mean()    
imp = imp.sort_values(ascending=False)

In [19]:
abun_imp_features_patient = imp[:10]

In [20]:
abun_imp_features_patient

1
1262824|Clostridium_sp_CAG_58              0.022051
360807|Roseburia_inulinivorans             0.022015
33038|Ruminococcus_gnavus                  0.021486
418240|Blautia_wexlerae                    0.021177
292800|Flavonifractor_plautii              0.020219
1550024|Ruthenibacterium_lactatiformans    0.019841
2108523|Lawsonibacter_asaccharolyticus     0.018167
28118|Odoribacter_splanchnicus             0.017737
853|Faecalibacterium_prausnitzii           0.015651
1681|Bifidobacterium_bifidum               0.015081
dtype: float64

# metabolomics

In [21]:
X = bioc_patient_data[:].transpose()
y = y_full[X.index].to_numpy()

ramdon_num_list = random.sample(range(1, 2000), 200)
imp_f_list = []
gdbt_list = []

for i in ramdon_num_list:
    GDBT = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                             learning_rate=0.05, loss='deviance', max_depth=5,
                             max_features=30, max_leaf_nodes=None,
                             min_impurity_decrease=0.0, min_impurity_split=None,
                             min_samples_leaf=9, min_samples_split=60,
                             min_weight_fraction_leaf=0.0, n_estimators=100,
                             n_iter_no_change=None, presort='deprecated',
                             random_state=i, subsample=1, tol=0.0001,
                             validation_fraction=0.1, verbose=0,
                             warm_start=False)
    gdbt_list.append(("%s" %(i), GDBT))
    GDBT.fit(X, y)
    imp = GDBT.feature_importances_
    imp_f_list.append(imp)
    
imp = pd.DataFrame(imp_f_list, columns=X.columns).mean()    
imp = imp.sort_values(ascending=False)

In [22]:
bioc_imp_features_patient = imp[:10]

In [23]:
bioc_imp_features_patient

cholesterol                                     0.044102
X - 21442                                       0.038969
X - 25172                                       0.038627
guanidinosuccinate                              0.032042
sphingomyelin (d18:2/21:0, d16:2/23:0)*         0.023786
1-stearoyl-2-docosahexaenoyl-GPC (18:0/22:6)    0.023036
serotonin                                       0.022485
3-(4-hydroxyphenyl)lactate (HPLA)               0.021332
betaine                                         0.021072
X - 24545                                       0.021063
dtype: float64

# Kegg gene 

In [24]:
X = kegg_patient_data[:].transpose()
y = y_full[X.index].to_numpy()

ramdon_num_list = random.sample(range(1, 2000), 200)
imp_f_list = []
gdbt_list = []
for i in ramdon_num_list:
    GDBT = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                             learning_rate=0.1, loss='deviance', max_depth=4,
                             max_features=10, max_leaf_nodes=None,
                             min_impurity_decrease=0.0, min_impurity_split=None,
                             min_samples_leaf=7, min_samples_split=10,
                             min_weight_fraction_leaf=0.0, n_estimators=100,
                             n_iter_no_change=None, presort='deprecated',
                             random_state=i, subsample=0.7, tol=0.0001,
                             validation_fraction=0.1, verbose=0,
                             warm_start=False)
    gdbt_list.append(("%s" %(i), GDBT))
    GDBT.fit(X, y)
    imp = GDBT.feature_importances_
    imp_f_list.append(imp)
imp = pd.DataFrame(imp_f_list, columns=X.columns).mean()    
imp = imp.sort_values(ascending=False)

In [25]:
kegg_imp_features_patient = imp[:10]

In [26]:
kegg_imp_features_patient

K21576    0.008022
K00254    0.005057
K09740    0.004571
K23393    0.004499
K21395    0.004467
K21577    0.004362
K21417    0.004241
K07149    0.004174
K15724    0.004164
K21578    0.003913
dtype: float64

# Onset

In [33]:
y_full = meta_data.loc['illness_duration', :]
model_list = load("%s/result/classification/onset_models_dictionary.joblib" %(root))

# abundance

In [34]:
X = rela_abun_data[:].transpose()
y = y_full[X.index].to_numpy()

ramdon_num_list = random.sample(range(1, 2000), 200)
rf_imp_f_list = []
rf_list = []

for i in ramdon_num_list:    
    RF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=4, max_features=5,
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=20,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=False, random_state=i,
                         verbose=0, warm_start=False)
    rf_list.append(("%s" %(i), RF))
    RF.fit(X, y)
    imp = RF.feature_importances_
    rf_imp_f_list.append(imp)
rf_imp = pd.DataFrame(rf_imp_f_list, columns=X.columns).mean()    
rf_imp = rf_imp.sort_values(ascending=False)

In [35]:
abun_imp_features_onset = rf_imp[:10]

# Metabolomics

In [36]:
X = bioc_onset_data[:].transpose()
y = y_full[X.index].to_numpy()

ramdon_num_list = random.sample(range(1, 2000), 200)
rf_imp_f_list = []
rf_list = []

for i in ramdon_num_list:
    RF =  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='entropy', max_depth=5, max_features=5,
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=3, min_samples_split=6,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=False, random_state=i,
                         verbose=0, warm_start=False)
    rf_list.append(("%s" %(i), RF))
    RF.fit(X, y)
    imp = RF.feature_importances_
    rf_imp_f_list.append(imp)
rf_imp = pd.DataFrame(rf_imp_f_list, columns=X.columns).mean()    
rf_imp = rf_imp.sort_values(ascending=False)

In [37]:
bioc_imp_features_onset = rf_imp[:10]

# Kegg gene

In [38]:
X = kegg_onset_data[:].transpose()
y = y_full[X.index].to_numpy()

ramdon_num_list = random.sample(range(1, 2000), 200)
rf_imp_f_list = []
rf_list = []

for i in ramdon_num_list:
    RF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=7, max_features=40,
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=40,
                         min_weight_fraction_leaf=0.0, n_estimators=100,
                         n_jobs=None, oob_score=False, random_state=i,
                         verbose=0, warm_start=False)
    rf_list.append(("%s" %(i), RF))
    RF.fit(X, y)
    imp = RF.feature_importances_
    rf_imp_f_list.append(imp)
rf_imp = pd.DataFrame(rf_imp_f_list, columns=X.columns).mean()    
rf_imp = rf_imp.sort_values(ascending=False)

In [39]:
kegg_imp_features_onset = rf_imp[:10]

# write all imp features

In [60]:
imp_table = pd.DataFrame(0, index=range(1,11), columns=[0]).drop(0, 1)

In [71]:
imp_table.loc[:,'patient_abun_name'] = abun_imp_features_patient.index
imp_table.loc[:,'patient_abun_importance'] = abun_imp_features_patient.ravel()
imp_table.loc[:,'onset_abun_name'] = abun_imp_features_onset.index
imp_table.loc[:,'onset_abun_importance'] = abun_imp_features_onset.ravel()

In [73]:
imp_table.loc[:,'patient_bioc_name'] = bioc_imp_features_patient.index
imp_table.loc[:,'patient_bioc_importance'] = bioc_imp_features_patient.ravel()
imp_table.loc[:,'onset_bioc_name'] = bioc_imp_features_onset.index
imp_table.loc[:,'onset_bioc_importance'] = bioc_imp_features_onset.ravel()

In [74]:
imp_table.loc[:,'patient_kegg_name'] = kegg_imp_features_patient.index
imp_table.loc[:,'patient_kegg_importance'] = kegg_imp_features_patient.ravel()
imp_table.loc[:,'onset_kegg_name'] = kegg_imp_features_onset.index
imp_table.loc[:,'onset_kegg_importance'] = kegg_imp_features_onset.ravel()

In [76]:
#imp_table.to_csv("%s/result/classification/important_features.csv" %(root))